Load model vào S7-1200

In [1]:
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as T
import numpy as np

from pymodbus.client import ModbusTcpClient

# 1. Tạo class map nếu bạn phân loại nhiều lớp, ví dụ rác
#    Giả sử 6 lớp: paper, glass, metal, plastic, cardboard, trash
CLASS_NAMES = ["paper", "glass", "metal", "plastic", "cardboard", "trash"]

# 2. Tải mô hình ResNet50 đã train
def load_model(model_path):
    # Khởi tạo ResNet50
    model = torch.hub.load('pytorch/vision', 'resnet50', pretrained=False)
    # model.fc = nn.Linear(2048, 6)  # Sửa số lớp đầu ra nếu cần (6 lớp)
    # Khởi tạo xong, load state_dict
    checkpoint = torch.load(model_path, map_location='cpu')
    model.load_state_dict(checkpoint)
    model.eval()
    return model

# 3. Kết nối tới PLC S7-1200 qua Modbus TCP/IP
#    Thông thường S7-1200 là Server (Slave), PLC_IP là địa chỉ IP, port = 502
PLC_IP = "192.168.0.1"
PLC_PORT = 502
plc_client = ModbusTcpClient(PLC_IP, port=PLC_PORT)

def send_to_plc(code):
    """
    Ghi giá trị `code` vào thanh ghi 0 (Holding Register)
    """
    if plc_client.connect():
        write_response = plc_client.write_registers(address=0, values=[code])
        if write_response and not write_response.isError():
            print(f"Sent code {code} to PLC successfully!")
        else:
            print("Error writing to PLC:", write_response)
        plc_client.close()
    else:
        print("Failed to connect PLC")

# 4. Hàm nhận diện
def classify_image(model, frame):
    """
    - Chuyển frame BGR từ OpenCV sang tensor
    - Normalization / Resize cho ResNet
    - Đẩy qua model -> lấy kết quả
    """
    transform = T.Compose([
        T.ToPILImage(),
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406],  # ResNet mean
                    std=[0.229, 0.224, 0.225])   # ResNet std
    ])

    img = transform(frame)
    img = img.unsqueeze(0)  # batch dimension [1, 3, 224, 224]

    with torch.no_grad():
        outputs = model(img)
        # Dạng [batch, num_classes]
        _, predicted = torch.max(outputs, 1)
        class_idx = predicted.item()

    return class_idx

def main():
    model_path = "ResnetModel.pth"
    model = load_model(model_path)
    print("Model loaded successfully.")

    # 5. Mở camera
    cap = cv2.VideoCapture(0)  # camera 0
    if not cap.isOpened():
        print("Could not open camera")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame from camera. Exiting...")
            break

        # 5.1) Infer
        class_idx = classify_image(model, frame)
        class_name = CLASS_NAMES[class_idx]
        print("Detected:", class_name)

        # 5.2) Gửi code sang PLC
        # Mỗi lớp map sang 1 code: paper=1, glass=2, ...
        class_map = {
            "paper": 1,
            "glass": 2,
            "metal": 3,
            "plastic": 4,
            "cardboard": 5,
            "trash": 6
        }
        code = class_map[class_name]
        send_to_plc(code)

        # 5.3) Hiển thị
        cv2.putText(frame, class_name, (10,30), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0,255,0), 2, cv2.LINE_AA)
        cv2.imshow("ResNet Real-time", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Cleanup
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Using cache found in C:\Users\PC/.cache\torch\hub\pytorch_vision_main
c:\Users\PC\Desktop\pythonProject2\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\PC\Desktop\pythonProject2\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\PC\AppData\Local\Temp\ipykernel_19248\4190325042.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untruste

TypeError: Expected state_dict to be dict-like, got <class 'torch.jit._script.RecursiveScriptModule'>.